# Proyecto final de la asignatura Minería de datos y el paradigma Big Data - curso 2022/23

## Imports

In [2]:
import pandas as pd
import numpy as np


## Rutas

In [3]:
path = './Data/'
file = 'itineraries_300000_clean.csv'

## Limpieza del dataset

In [26]:
finalFile = 'itineraries_clean.csv'

columnsSelected = [ 'legId', 'searchDate', 'flightDate', 'startingAirport',
    'destinationAirport', 'travelDuration',
    'isBasicEconomy', 'totalFare',
    'seatsRemaining', 'totalTravelDistance', 
    'segmentsAirlineName', 'segmentsAirlineCode']

data = pd.read_csv(path + file,usecols=columnsSelected,index_col= None)
print(data)
data.to_csv(path + finalFile,index=False)

                                   legId  searchDate  flightDate  \
0       9ca0e81111c683bec1012473feefd28f  2022-04-16  2022-04-17   
1       98685953630e772a098941b71906592b  2022-04-16  2022-04-17   
2       98d90cbc32bfbb05c2fc32897c7c1087  2022-04-16  2022-04-17   
3       969a269d38eae583f455486fa90877b4  2022-04-16  2022-04-17   
4       980370cf27c89b40d2833a1d5afc9751  2022-04-16  2022-04-17   
...                                  ...         ...         ...   
299995  90591f3aeeba1c88e66c64746299b28c  2022-04-17  2022-05-11   
299996  bc251918565f2c1aff1ea039220cfece  2022-04-17  2022-05-11   
299997  b6d15042b341b0845241af77866b97d0  2022-04-17  2022-05-11   
299998  55efe31162876477d2761abd5abb438d  2022-04-17  2022-05-11   
299999  e22ff727fea0f61ba78212ecc49b1992  2022-04-17  2022-05-11   

       startingAirport destinationAirport travelDuration  isBasicEconomy  \
0                  ATL                BOS        PT2H29M           False   
1                  ATL         

## Lectura del dataset

In [49]:
split = lambda x: x.split("||")

data = pd.read_csv(path + file,decimal='.',usecols=columnsSelected,parse_dates=[1,2], converters={'segmentsAirlineName':split, 'segmentsAirlineCode':split})
arrayNp = data.to_numpy()
print(data.columns)
data[:10]

Index(['legId', 'searchDate', 'flightDate', 'startingAirport',
       'destinationAirport', 'travelDuration', 'isBasicEconomy', 'totalFare',
       'seatsRemaining', 'totalTravelDistance', 'segmentsAirlineName',
       'segmentsAirlineCode'],
      dtype='object')


,legId,searchDate,flightDate,startingAirport,destinationAirport,travelDuration,isBasicEconomy,totalFare,seatsRemaining,totalTravelDistance,segmentsAirlineName,segmentsAirlineCode
0,9ca0e81111c683bec1012473feefd28f,2022-04-16,2022-04-17,ATL,BOS,PT2H29M,False,248.6,9,947.0,[Delta],[DL]
1,98685953630e772a098941b71906592b,2022-04-16,2022-04-17,ATL,BOS,PT2H30M,False,248.6,4,947.0,[Delta],[DL]
2,98d90cbc32bfbb05c2fc32897c7c1087,2022-04-16,2022-04-17,ATL,BOS,PT2H30M,False,248.6,9,947.0,[Delta],[DL]
3,969a269d38eae583f455486fa90877b4,2022-04-16,2022-04-17,ATL,BOS,PT2H32M,False,248.6,8,947.0,[Delta],[DL]
4,980370cf27c89b40d2833a1d5afc9751,2022-04-16,2022-04-17,ATL,BOS,PT2H34M,False,248.6,9,947.0,[Delta],[DL]
5,79eda9f841e226a1e2121d74211e595c,2022-04-16,2022-04-17,ATL,BOS,PT2H38M,False,248.6,7,947.0,[JetBlue Airways],[B6]
6,9335fae376c38bb61263281779f469ec,2022-04-16,2022-04-17,ATL,BOS,PT4H12M,False,251.1,3,956.0,"[American Airlines, American Airlines]","[AA, AA]"
7,3904bf87f2d1daf334f1ae7e3b876028,2022-04-16,2022-04-17,ATL,BOS,PT5H18M,False,251.1,3,956.0,"[American Airlines, American Airlines]","[AA, AA]"
8,d93988734c44a3c075d9efe373352507,2022-04-16,2022-04-17,ATL,BOS,PT5H32M,False,251.1,7,956.0,"[American Airlines, American Airlines]","[AA, AA]"
9,562e7d5dd6ecbf1509c0c19711dbdca9,2022-04-16,2022-04-17,ATL,BOS,PT6H38M,False,251.1,7,956.0,"[American Airlines, American Airlines]","[AA, AA]"


## Dividir datos en meses

In [58]:

enero = data[data['flightDate'].dt.month == 1]
febrero = data[data['flightDate'].dt.month == 2]# && data['flightDate']<=pd.to_datetime('2022-02-28')]
marzo = data[data['flightDate'].dt.month == 3]# && data['flightDate']<=pd.to_datetime('2022-03-31')]
abril = data[data['flightDate'].dt.month == 4]# && data['flightDate']<=pd.to_datetime('2022-04-30')]
mayo = data[data['flightDate'].dt.month == 5]# && data['flightDate']<=pd.to_datetime('2022-05-31')]
junio = data[data['flightDate'].dt.month == 6]
julio = data[data['flightDate'].dt.month == 7]# && data['flightDate']<=pd.to_datetime('2022-07-31')]
agosto = data[data['flightDate'].dt.month == 8]# && data['flightDate']<=pd.to_datetime('2022-08-31')]
septiembre = data[data['flightDate'].dt.month == 9]# && data['flightDate']<=pd.to_datetime('2022-09-30')]
octubre  = data[data['flightDate'].dt.month == 10]# && data['flightDate']<=pd.to_datetime('2022-10-31')]
noviembre = data[data['flightDate'].dt.month == 11]# && data['flightDate']<=pd.to_datetime('2022-11-30')]
diciembre = data[data['flightDate'].dt.month == 12]# && data['flightDate']<=pd.to_datetime('2022-12-31')]


In [ ]:
enero.to_csv(path + "enero.csv",index=False)
febrero.to_csv(path + "febrero.csv",index=False)
marzo.to_csv(path + "marzo.csv",index=False)
abril.to_csv(path + "abril.csv",index=False)
mayo.to_csv(path + "mayo.csv",index=False)
junio.to_csv(path + "junio.csv",index=False)
julio.to_csv(path + "julio.csv",index=False)
agosto.to_csv(path + "agosto.csv",index=False)
septiembre.to_csv(path + "septiembre.csv",index=False)
octubre.to_csv(path + "octubre.csv",index=False)
noviembre.to_csv(path + "noviembre.csv",index=False)
diciembre.to_csv(path + "diciembre.csv",index=False)

In [63]:
MaxDestinosMes = [
    enero['destinationAirport'].max(),
    febrero['destinationAirport'].max(),
    marzo['destinationAirport'].max(),
    abril['destinationAirport'].max(),
    mayo['destinationAirport'].max(),
    junio['destinationAirport'].max(),
    agosto['destinationAirport'].max(),
    septiembre['destinationAirport'].max(),
    octubre['destinationAirport'].max(),
    noviembre['destinationAirport'].max(),
    diciembre['destinationAirport'].max(),
]
MaxDestinosMes

[nan, nan, nan, 'SFO', 'SFO', nan, nan, nan, nan, nan, nan]

## Mismos vuelos en distintas compañias